In [1]:
import findspark
findspark.init()

import os
print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME'])

/usr/lib/jvm/java-11-openjdk-amd64
/usr/local/spark


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

from delta import *

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]
warehouse_dir = f"{work_dir}/spark-warehouse"

# Create spark session with hive enabled
builder = (
    SparkSession
    .builder
    .appName("pyspark-notebook")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "true")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .enableHiveSupport()
)

In [3]:
# 델타 레이크 생성시에 반드시 `configure_spark_with_delta_pip` 구성을 통해 실행되어야 정상적인 델타 의존성이 로딩됩니다
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark.conf.set("spark.sql.decimalOperations.allowPrecisionLoss", "true")
spark

In [5]:
def show(queries, num_rows = 20):
    for query in queries.split(";"):
        spark.sql(query).show(num_rows, truncate=False)

def sql(query):
    return spark.sql(query)

def history(dbName, tableName):
    return spark.sql("describe history {}.{}".format(dbName, tableName))

def table(dbName, tableName):
    return spark.read.format("delta").table("{}.{}".format(dbName, tableName))

def describe(dbName, tableName, extended = True, num_rows = 20):
    if extended:
        show("describe extended {}.{}".format(dbName, tableName), num_rows)
    else:
        show("describe {}.{}".format(dbName, tableName), num_rows)

def ls(target):
    !ls -al {target}

def ls_and_head(target, lineno):
    !ls -al {target} | grep -v 'crc' | head -{lineno}

def cat(filename):
    !cat {filename}

def grep(keyword, filename):
    !grep -i {keyword} {filename}

def grep_and_json(keyword, filename):
    !grep {keyword} {filename} | python -m json.tool

def grep_sed_json(keyword, lineno, filename):
    !grep {keyword} {filename} | sed -n {lineno}p | python -m json.tool


In [6]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

def dropAndRemoveTable(dbName, tableName):
    location="/home/jovyan/work/spark-warehouse/{}".format(tableName)
    !rm -rf {location}
    sql("DROP TABLE IF EXISTS {}.{}".format(dbName, tableName))

In [8]:
dbName = "taxidb"
sourceTable = "tripAggregates"
targetTable = "streamTarget"

In [9]:
source=f"spark-warehouse/{dbName}.db/{sourceTable}"
target=f"spark-warehouse/{dbName}.db/{targetTable}"
chkpnt=f"{target}/_checkpoint"
ls_and_head(source, 10)

stream_df = spark.readStream.format("delta").load(source)
stream_df = stream_df.withColumn("RecordStreamTime", current_timestamp())
stream_df.printSchema()

total 20
drwxrwxrwx 1 jovyan 1000  512 Nov  5 08:05 .
drwxrwxrwx 1 jovyan 1000  512 Nov  5 08:34 ..
drwxrwxrwx 1 jovyan 1000  512 Nov  5 08:05 _change_data
drwxrwxrwx 1 jovyan 1000  512 Nov  5 08:05 _delta_log
-rwxrwxrwx 1 jovyan 1000 1185 Nov  5 08:05 part-00000-0b48e892-6353-4d8d-b156-d4aa897125e4.c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  968 Nov  5 08:05 part-00000-17099711-3cc6-4848-a0dc-00b47f5c4d8d-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  968 Nov  5 08:05 part-00000-465ec638-994d-42a1-831e-3cd9b2d37642-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  968 Nov  5 08:02 part-00000-6330f8a7-eed5-4186-903b-639d06fe7cec-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  968 Nov  5 08:00 part-00000-7cd2c96a-b44a-431b-8691-11d38d341a59-c000.snappy.parquet
root
 |-- VendorId: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- FareAmount: integer (nullable = true)
 |-- RecordStreamTime: timestamp (nullable = false)



In [12]:
select_columns = [
'VendorId', 'PassengerCount', 'FareAmount', 'RecordStreamTime'
]
stream_df = stream_df.select(select_columns)
stream_query = stream_df.writeStream.format("delta").option("checkpointLocation", chkpnt).start(target)

AttributeError: 'StreamingQuery' object has no attribute 'awaitTerminaion'

In [13]:
stream_query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [20]:
spark.sql(f"describe extended delta.`/home/jovyan/work/{target}`")

col_name,data_type,comment
VendorId,int,
PassengerCount,int,
FareAmount,int,
RecordStreamTime,timestamp,
,,
# Partitioning,,
Not partitioned,,
,,
# Detailed Table Information,,
Name,delta.`file:/home/jovyan/work/spark-warehouse/taxidb.db/streamTarget`,
